In [2]:
import pandas as pd
from tabulate import tabulate
import requests, json, sys, os, re

# Ensure the utils package is installed and accessible
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#import local packages
from plumbing.natstat_ingestion import ingest_teams_data, ingest_games_data, parse_game_data

In [3]:
# TEST URL OUTPUT
# url = f"https://interst.at/game/pfb/2024"
# response = requests.get(url)
# response.raise_for_status()
# data = response.json()
# pretty_data = json.dumps(data, indent=3)
# #print(pretty_data)

+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
game_df = ingest_games_data()
team_df = ingest_teams_data()

In [11]:
game_df = game_df[game_df['status'] == 'Final']
print(tabulate(team_df, headers='keys', tablefmt='grid'))

TypeError: 'Styler' object is not iterable

In [4]:
#TODO: Write a games loop that loops through all (active only?) games in game_df and calls the (game and player) parsing functions

from plumbing.natstat_ingestion import ingest_teams_data, ingest_games_data, parse_game_data

# INGEST
team_df = ingest_teams_data()
game_df = ingest_games_data()

team_df = team_df.style.background_gradient(cmap='viridis')

# Get list of finished games and their URLs
final_games_df = game_df[game_df['status'] == 'Final']
finished_games_urls_list = final_games_df['game_api_url'].tolist()
# Initialize game_stat and player_stat dataframes to append each game to 
game_stat_df = pd.DataFrame()
player_stat_df = pd.DataFrame()
error_log = []

for game_url in finished_games_urls_list:
    print(f"Game URL: {game_url}")
    try:
        # Append the parsed game and player data to the respective dataframes
        game_data, player_data = parse_game_data(game_url)
        print(player_data)
        game_stat_df = pd.concat([game_stat_df, game_data], ignore_index=True)
        
        # player_data = parse_player_data(data)
        # player_stat_df = pd.concat([player_stat_df, player_data], ignore_index=True)
    except requests.RequestException as e:
        print(f"Error with URL {game_url}: {e}")
        error_log.append((game_url, str(e)))
    
# Print or save the error log for further inspection
print("Error Log:", error_log)


Game URL: https://interst.at/game/pfb/16580
   player_id      player_name         team position starter
0   47622835    Lamar Jackson    Baltimore       QB       N
1   47621015    Derrick Henry    Baltimore       RB       N
2   52244751      Zay Flowers    Baltimore       WR       N
3   47623023     Justice Hill    Baltimore       RB       N
4   47773702    Isaiah Likely    Baltimore       TE       N
..       ...              ...          ...      ...     ...
56  48206172   Nazeeh Johnson  Kansas City        S       N
57  47776304  Joshua Williams  Kansas City       CB       N
58  47620495   Mecole Hardman  Kansas City       WR       N
59  47620657  Harrison Butker  Kansas City        K       N
60  57967666      Matt Araiza  Kansas City        P       N

[61 rows x 5 columns]
Game URL: https://interst.at/game/pfb/16581
   player_id           player_name                 team position starter
0   47624496           Jordan Love    Green Bay Packers       QB       N
1   47819021          M

In [6]:
print(tabulate(game_stat_df, headers='keys', tablefmt='grid'))

+----+-----------+------------+-------------+----------+----------------+-----------------+---------------+--------------+---------------------------------+--------------+
|    |   game_id | gameday    | starttime   | status   | visitor_team   |   visitor_score | home_team     |   home_score | venue                           |   attendance |
+====+===========+============+=============+==========+================+=================+===============+==============+=================================+==============+
|  0 |     16580 | 2024-09-05 | 20:40       | Final    | Baltimore      |              20 | Kansas City   |           27 | GEHA Field at Arrowhead Stadium |        73611 |
+----+-----------+------------+-------------+----------+----------------+-----------------+---------------+--------------+---------------------------------+--------------+
|  1 |     16581 | 2024-09-06 | 20:15       | Final    | Green Bay      |              29 | Philadelphia  |           34 | Corinthians Arena

In [ ]:
# def enrich_games_data(games_df):
#     """Takes a dataframe of games and parses the data for each game.

#     Args:
#         game_url (_type_): _description_
#     """
#     # Get list of finished games and their URLs
#     final_games_df = game_df[game_df['status'] == 'Final']
#     finished_games_urls_list = final_games_df['game_api_url'].tolist()

#     # Initialize game_stat and player_stat dataframes to append each game to 
#     game_stat_df = pd.DataFrame()
#     player_stat_df = pd.DataFrame()

#     for game_url in finished_games_urls_list:
#         game_url = f"https://interst.at{game_url}"
#         response = requests.get(game_url)
#         response.raise_for_status()
#         data = response.json()
#         # Append the parsed game and player data to the respective dataframes
#         pd.concat([game_stat_df, parse_game_data(data)], ignore_index=True)
#         pd.concat([player_stat_df, parse_player_data(data)], ignore_index=True)
    
#     return game_stat_df, player_stat_df

# gamestat_df, playerstat_df = enrich_games_data(data)